In [0]:
%run "../../../config/setup"

In [0]:
CREATE OR REPLACE TABLE customer_profile_360_gold AS
WITH favorite_item AS (
  SELECT 
    c.customer_id,
    FIRST(i.item_name) AS favorite_item
  FROM customers_silver c
  LEFT JOIN transactions_silver t ON c.customer_id = t.customer_id
  LEFT JOIN orders_silver o ON t.transaction_id = o.transaction_id
  LEFT JOIN items_dedupped i ON o.item_id = i.item_id
  GROUP BY c.customer_id
  ORDER BY COUNT(i.item_name) DESC
)
SELECT
  c.customer_id,
  c.first_name,
  c.last_name,
  ANY_VALUE(c.city) AS city,
  MAX(YEAR(c.registration_date)) AS registration_year,
  COALESCE(SUM(t.total_amount), 0) AS total_lifetime_spend,
  AVG(t.total_amount) AS avg_order_value,
  MAX(t.transaction_date) AS last_purchase_date,
  MODE(p.pref_value) AS preferred_category,
  MODE(f.interaction_type) AS top_facebook_action,
  COUNT(cl.action) AS total_clicks,
  fi.favorite_item,
  EXISTS(SELECT 1 FROM clickstream_customers_silver cl2 WHERE cl2.customer_id = c.customer_id AND cl2.action LIKE '%like%') AS like_engagement
FROM customers_silver c
LEFT JOIN transactions_silver t ON c.customer_id = t.customer_id
LEFT JOIN preferences_silver p ON c.customer_id = p.customer_id AND p.preference_category = 'product_type'
LEFT JOIN facebook_silver f ON c.customer_id = f.customer_id
LEFT JOIN clickstream_customers_silver cl ON c.customer_id = cl.customer_id AND cl.action = 'page_view'
LEFT JOIN favorite_item fi ON c.customer_id = fi.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, fi.favorite_item;